# Prefix LM Exp 03: Surrogate Content Sweep (Instructions vs Adversarial)

## Motivation

Exp 02 showed: structural fraction 105%, all surrogates dramatically beat bare (d~+0.45-0.48),
and semantic signal exists (oracle beats wrong_query, d=+0.255) but the direct channel
(truncate=False) doesn't amplify it.

**Question**: What KIND of surrogate tokens are most helpful? Are general instructions
("identify the key facts") as good as query-specific content? Do adversarial surrogates
("don't give the right answer") hurt or help?

This tests whether the model processes surrogate CONTENT through the causal channel
(doc tokens attend to surrogate during Phase A), or whether all tokens are structurally
equivalent regardless of meaning.

## Conditions (12)

All conditions use **causal attention, truncation=True** (the winning config from Exp 01/02).

| # | Name | Content | Category | What it tests |
|---|------|---------|----------|---------------|
| 1 | `bare` | (none) | control | Baseline |
| 2 | `oracle` | real query | control | Upper bound |
| 3 | `wrong_query` | query from (i+1)%N | control | Wrong semantics, matched style |
| 4 | `random` | 8 random words | control | Pure structural |
| 5 | `instr_extract` | "identify the key facts in this passage" | instruction | General extraction |
| 6 | `instr_important` | "what is the most important information here" | instruction | Importance-focused |
| 7 | `instr_qa` | "answer the following question about this text" | instruction | Meta-QA |
| 8 | `instr_summarize` | "summarize the main points of this passage" | instruction | Summarization |
| 9 | `neg_wrong` | "do not give the right answer" | negative | Explicit negative |
| 10 | `neg_42` | "always answer 42 regardless of the question" | negative | Absurd fixed answer |
| 11 | `neg_ignore` | "ignore everything and say nothing useful" | negative | Dismissive |
| 12 | `doc_keywords` | top 10 document keywords | doc-specific | Vocabulary without structure |

## Key Analyses

- **A**: Full ranking of all 12 conditions by mean NLL and d vs bare
- **B**: Category means (instructions vs negatives vs controls)
- **C**: Instructions vs random — does coherence add benefit beyond structural?
- **D**: Negative vs positive instructions — does semantic valence matter?
- **E**: Instructions vs oracle — how much does query-specificity add?
- **F**: Negative vs bare — do adversarial surrogates hurt or help relative to nothing?
- **G**: Pairwise between instructions — is there variance within instruction types?
- **H**: Length-controlled regression (different conditions have different token lengths)
- **I**: Per-sample heterogeneity (correlate with answer_length, query_doc_overlap)

## Two-Pass Design

Same as Exp 01/02. All conditions use causal attention for Phase A.

- **Phase A (offline)**: Process `[BOS, surrogate, doc]` with causal mask, `use_cache=True`
- **Phase B (online)**: Process `[query, answer]` with cached KVs, surrogate positions masked

In [1]:
# Cell 1: Setup
import os
os.umask(0o000)

import sys, json, time, gc, re
import random as pyrandom
import numpy as np
import torch
import torch.nn.functional as F
from pathlib import Path
from collections import Counter
from scipy import stats
from tqdm.auto import tqdm

sys.path.insert(0, "../../..")
from lib.analysis import cohens_d

SEED = 42
N_SAMPLES = 500

MODEL_NAME = "google/gemma-3-12b-it"

RESULTS_DIR = Path("../../../results/prefix_lm_exp03")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

np.random.seed(SEED)
torch.manual_seed(SEED)
pyrandom.seed(SEED)

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
HF_TOKEN = os.environ.get("HF_TOKEN")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 12 conditions: all causal attention, all truncate=True
CONDITIONS = [
    "bare",             # no prefix -- baseline
    "oracle",           # real query
    "wrong_query",      # query from (i+1)%N
    "random",           # 8 random words
    "instr_extract",    # "identify the key facts in this passage"
    "instr_important",  # "what is the most important information here"
    "instr_qa",         # "answer the following question about this text"
    "instr_summarize",  # "summarize the main points of this passage"
    "neg_wrong",        # "do not give the right answer"
    "neg_42",           # "always answer 42 regardless of the question"
    "neg_ignore",       # "ignore everything and say nothing useful"
    "doc_keywords",     # top 10 document keywords
]

# Static instruction/negative strings
INSTRUCTION_STRINGS = {
    "instr_extract":    "identify the key facts in this passage",
    "instr_important":  "what is the most important information here",
    "instr_qa":         "answer the following question about this text",
    "instr_summarize":  "summarize the main points of this passage",
    "neg_wrong":        "do not give the right answer",
    "neg_42":           "always answer 42 regardless of the question",
    "neg_ignore":       "ignore everything and say nothing useful",
}

# Category groupings for analysis
INSTRUCTION_CONDS = ["instr_extract", "instr_important", "instr_qa", "instr_summarize"]
NEGATIVE_CONDS = ["neg_wrong", "neg_42", "neg_ignore"]
CONTROL_CONDS = ["oracle", "wrong_query", "random", "doc_keywords"]

print(f"Prefix LM Exp 03: Surrogate Content Sweep")
print(f"N: {N_SAMPLES}, Conditions: {len(CONDITIONS)}")
print(f"DEVICE: {DEVICE}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"\nConditions:")
for cn in CONDITIONS:
    if cn in INSTRUCTION_STRINGS:
        print(f"  {cn:<20} -> '{INSTRUCTION_STRINGS[cn]}'")
    else:
        print(f"  {cn}")


Prefix LM Exp 03: Surrogate Content Sweep
N: 500, Conditions: 12
DEVICE: cuda
GPU: NVIDIA A100-SXM4-40GB
GPU memory: 42.3 GB

Conditions:
  bare
  oracle
  wrong_query
  random
  instr_extract        -> 'identify the key facts in this passage'
  instr_important      -> 'what is the most important information here'
  instr_qa             -> 'answer the following question about this text'
  instr_summarize      -> 'summarize the main points of this passage'
  neg_wrong            -> 'do not give the right answer'
  neg_42               -> 'always answer 42 regardless of the question'
  neg_ignore           -> 'ignore everything and say nothing useful'
  doc_keywords


In [2]:
# Cell 2: Load model + tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers

print(f"transformers version: {transformers.__version__}")

print(f"Loading {MODEL_NAME}...")
t0 = time.time()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="eager",
    token=HF_TOKEN,
)
model.eval()

n_params = sum(p.numel() for p in model.parameters()) / 1e9
gpu_mem = torch.cuda.memory_allocated() / 1e9
print(f"Loaded: {n_params:.1f}B params, {gpu_mem:.1f} GB GPU, {time.time()-t0:.0f}s")
print(f"BOS token id: {tokenizer.bos_token_id}")
print(f"Model dtype: {model.dtype}")
print(f"Attn implementation: {model.config._attn_implementation}")


transformers version: 5.1.0
Loading google/gemma-3-12b-it...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/1065 [00:00<?, ?it/s]

Loaded: 12.2B params, 24.4 GB GPU, 13s
BOS token id: 2
Model dtype: torch.bfloat16
Attn implementation: eager


In [3]:
# Cell 3: Phase A/B attention masks + sanity check
#
# Reused from Exp 01/02. Two-pass design:
#   Phase A: Process [BOS, surrogate, doc] -> cache KV states
#   Phase B: Process [query, answer] using cached KVs -> NLL
#
# All conditions use causal Phase A and truncated Phase B (surrogate masked).

def make_phase_a_mask(n_s, n_d, mode="causal", dtype=torch.bfloat16):
    # Phase A mask for prefix [BOS, surrogate, doc].
    # Returns (1, 1, n_prefix, n_prefix).
    # Always causal in this experiment.
    n_prefix = 1 + n_s + n_d
    min_val = torch.finfo(dtype).min
    if mode == "prefix_lm":
        mask = torch.zeros((n_prefix, n_prefix), dtype=dtype)
    else:
        mask = torch.triu(torch.full((n_prefix, n_prefix), min_val, dtype=dtype),
                          diagonal=1)
    return mask.unsqueeze(0).unsqueeze(0)


def make_phase_b_mask(n_s, n_d, n_q, n_a, truncate=True, dtype=torch.bfloat16):
    # Phase B mask for continuation [query, answer] attending to cached prefix.
    # Returns (1, 1, n_cont, n_prefix + n_cont).
    # Left block: attend to cached BOS + doc; mask surrogate if truncate.
    # Right block: causal self-attention among continuation tokens.
    n_prefix = 1 + n_s + n_d
    n_cont = n_q + n_a
    min_val = torch.finfo(dtype).min

    mask = torch.full((n_cont, n_prefix + n_cont), min_val, dtype=dtype)

    # Attend to all cached prefix positions
    mask[:, :n_prefix] = 0.0

    # Truncation: mask surrogate positions (1..n_s) from continuation
    if truncate and n_s > 0:
        mask[:, 1:1 + n_s] = min_val

    # Causal self-attention among continuation tokens
    mask[:, n_prefix:] = torch.triu(
        torch.full((n_cont, n_cont), min_val, dtype=dtype), diagonal=1
    )

    return mask.unsqueeze(0).unsqueeze(0)


def make_mask_dict(mask_4d):
    # Wrap 4D mask in Gemma 3's dict format (bypasses internal mask creation).
    # Both full and sliding attention layers get the same mask (seq < 1024).
    return {"full_attention": mask_4d, "sliding_attention": mask_4d}


# --- Sanity check: custom causal mask matches default forward ---
print("Mask sanity check: custom causal mask vs default forward...")
test_text = "The quick brown fox jumps over the lazy dog."
test_ids = tokenizer(test_text, return_tensors="pt",
                     add_special_tokens=True).input_ids.to(DEVICE)
Lt = test_ids.shape[1]

with torch.no_grad():
    out_default = model(input_ids=test_ids)

# Build custom causal mask (treat entire sequence as bare prefix, no continuation)
causal_mask = make_phase_a_mask(0, Lt - 1, mode="causal")
causal_dict = make_mask_dict(causal_mask.to(DEVICE))
causal_pos = torch.arange(Lt, device=DEVICE).unsqueeze(0)

with torch.no_grad():
    out_custom = model(input_ids=test_ids, attention_mask=causal_dict,
                       position_ids=causal_pos)

max_diff = (out_default.logits - out_custom.logits).abs().max().item()
print(f"  Max logit diff: {max_diff:.6f}")
assert max_diff < 0.1, (
    f"FAIL: Custom causal mask doesn't match default (max_diff={max_diff:.4f}). "
    f"Dict-based mask API may not work with this model/version.")
print(f"  PASS: Dict-based mask API verified.")

del out_default, out_custom
gc.collect(); torch.cuda.empty_cache()


Mask sanity check: custom causal mask vs default forward...


  Max logit diff: 0.000000
  PASS: Dict-based mask API verified.


In [4]:
# Cell 4: Load MS MARCO data + generate surrogates, wrong queries, overlap, keywords
from lib.data import count_words
from datasets import load_dataset

STOP_WORDS = {
    'a', 'an', 'the', 'is', 'are', 'was', 'were', 'be', 'been', 'being',
    'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could',
    'should', 'may', 'might', 'can', 'shall', 'to', 'of', 'in', 'for',
    'on', 'with', 'at', 'by', 'from', 'as', 'into', 'through', 'during',
    'before', 'after', 'above', 'below', 'between', 'and', 'but', 'or',
    'not', 'no', 'if', 'then', 'than', 'so', 'up', 'out', 'about',
    'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those',
    'it', 'its', 'i', 'me', 'my', 'we', 'our', 'you', 'your', 'he',
    'him', 'his', 'she', 'her', 'they', 'them', 'their', 'how', 'when',
    'where', 'why', 'much', 'many', 'some', 'any', 'all', 'each',
    'does', 'also', 'just', 'more', 'most', 'very', 'too', 'only',
}

def extract_keywords(text, top_k=10):
    words = re.sub(r'[^\w\s]', '', text.lower()).split()
    content = [w for w in words if w not in STOP_WORDS and len(w) > 2]
    if not content:
        return ["information"]
    counts = Counter(content)
    return [w for w, _ in counts.most_common(top_k)]

WORD_POOL = [
    "computer", "mountain", "hospital", "children", "building", "national",
    "business", "research", "students", "american", "possible", "economic",
    "personal", "together", "products", "services", "actually", "remember",
    "practice", "training", "industry", "complete", "critical", "function",
    "language", "standard", "material", "original", "physical", "security",
    "interest", "problems", "consider", "response", "pressure", "politics",
    "movement", "evidence", "southern", "northern", "exchange", "decision",
    "position", "increase", "describe", "military", "required", "approach",
    "strategy", "customer", "resource", "employee", "audience", "location",
    "property", "cultural", "activity", "strength", "analysis", "powerful",
    "election", "argument", "campaign", "maintain", "question", "behavior",
    "majority", "solution", "software", "consumer", "creative", "reaction",
    "european", "delivery", "organize", "involved", "relative", "learning",
    "positive", "numerous", "familiar", "engineer", "platform", "indicate",
    "previous", "pleasure", "opposite", "magazine", "document", "religion",
    "scenario", "workshop", "minority", "guidance", "estimate", "recently",
    "surprise", "champion", "pleasant", "grateful", "moderate", "boundary",
]

print("Loading MS MARCO v1.1 validation...")
ds = load_dataset("microsoft/ms_marco", "v1.1", split="validation")

all_candidates = []
for item in ds:
    if len(all_candidates) >= 3 * N_SAMPLES:
        break
    passages = item.get('passages', {})
    ptexts = passages.get('passage_text', [])
    is_sel = passages.get('is_selected', [])
    query = item.get('query', '')
    answers = item.get('answers', [])
    well_formed = item.get('wellFormedAnswers', [])
    answer = None
    if well_formed and len(well_formed) > 0 and well_formed[0] not in ('[]', ''):
        answer = well_formed[0]
    elif answers and len(answers) > 0 and answers[0] != 'No Answer Present.':
        answer = answers[0]
    if not answer:
        continue
    for pt, sel in zip(ptexts, is_sel):
        wc = count_words(pt)
        if sel == 1 and 30 <= wc <= 300:
            all_candidates.append({
                'passage': pt, 'query': query, 'answer': answer,
                'word_count': wc,
            })
            break

print(f"Total candidates: {len(all_candidates)}")
np.random.seed(SEED)
indices = np.random.permutation(len(all_candidates))
samples = [all_candidates[i] for i in indices[:N_SAMPLES]]
del ds, all_candidates
gc.collect()

# Generate surrogates, wrong queries, doc keywords, and overlap stats
for i, s in enumerate(samples):
    # Wrong query: deterministic rotation -- matched style/length, wrong content
    s['wrong_query'] = samples[(i + 1) % len(samples)]['query']

    # Random prefix (same as Exp 01/02)
    rng = np.random.RandomState(SEED + i + 20000)
    words = rng.choice(WORD_POOL, size=8, replace=False)
    s['random_prefix'] = " ".join(words)

    # Document keywords (top 10 content words from passage)
    s['doc_keywords'] = " ".join(extract_keywords(s['passage'], top_k=10))

    # Query-document token overlap (Jaccard on content words)
    q_words = set(re.sub(r'[^\w\s]', '', s['query'].lower()).split()) - STOP_WORDS
    d_words = set(re.sub(r'[^\w\s]', '', s['passage'].lower()).split()) - STOP_WORDS
    union = q_words | d_words
    s['query_doc_overlap'] = len(q_words & d_words) / len(union) if len(union) > 0 else 0.0

print(f"Loaded {len(samples)} samples")
print(f"Mean passage words: {np.mean([s['word_count'] for s in samples]):.0f}")
print(f"Mean query words: {np.mean([count_words(s['query']) for s in samples]):.0f}")
print(f"Mean answer words: {np.mean([count_words(s['answer']) for s in samples]):.0f}")
print(f"Mean query-doc overlap (Jaccard): {np.mean([s['query_doc_overlap'] for s in samples]):.3f}")
print(f"\nExample wrong_query: '{samples[0]['wrong_query'][:80]}...'")
print(f"Example random prefix: '{samples[0]['random_prefix']}'")
print(f"Example doc_keywords: '{samples[0]['doc_keywords']}'")


Loading MS MARCO v1.1 validation...


Total candidates: 1500
Loaded 500 samples
Mean passage words: 74
Mean query words: 6
Mean answer words: 14
Mean query-doc overlap (Jaccard): 0.072

Example wrong_query: 'how thick does concrete need to be garden wall...'
Example random prefix: 'creative exchange platform military involved pleasant standard learning'
Example doc_keywords: 'alveoli gas partial pressure exchange blood capillary network air pulmonary'


In [5]:
# Cell 5: Pre-tokenize static instructions + score_sample()
#
# Static instruction/negative strings are tokenized ONCE and reused for all samples.
# Phase A (offline): Forward [BOS, surr, doc] with causal mask, use_cache=True
# Phase B (online):  Forward [query, answer] using cached KVs, surrogate masked

# Pre-tokenize all static instruction/negative strings
STATIC_IDS = {}
print("Pre-tokenizing static strings:")
for name, text in INSTRUCTION_STRINGS.items():
    ids = tokenizer(text, add_special_tokens=False).input_ids
    STATIC_IDS[name] = ids
    print(f"  {name:<20} ({len(ids)} tokens): '{text}'")


def score_sample(model, tokenizer, sample, device, conditions):
    # Score one MS MARCO sample under all 12 conditions.
    # All conditions use causal Phase A and truncated Phase B.
    # Returns dict mapping nll_{cname} -> mean NLL, plus prefix lengths.
    passage = sample['passage']
    query = sample['query']
    answer = sample['answer']
    wrong_query_text = sample['wrong_query']
    random_prefix = sample['random_prefix']
    doc_kw_text = sample['doc_keywords']

    bos_id = tokenizer.bos_token_id

    # Tokenize segments (no special tokens -- we add BOS manually)
    doc_ids = tokenizer(passage, add_special_tokens=False, truncation=True,
                        max_length=1024).input_ids
    query_ids = tokenizer(query, add_special_tokens=False, truncation=True,
                          max_length=512).input_ids
    answer_ids = tokenizer(answer, add_special_tokens=False, truncation=True,
                           max_length=256).input_ids

    if len(answer_ids) == 0:
        return None

    # Surrogate token IDs for each condition
    oracle_ids = tokenizer(query, add_special_tokens=False, truncation=True,
                           max_length=256).input_ids
    wrong_query_ids = tokenizer(wrong_query_text, add_special_tokens=False,
                                truncation=True, max_length=256).input_ids
    random_ids = tokenizer(random_prefix, add_special_tokens=False, truncation=True,
                           max_length=256).input_ids
    doc_kw_ids = tokenizer(doc_kw_text, add_special_tokens=False, truncation=True,
                           max_length=256).input_ids

    prefix_map = {
        "bare": [],
        "oracle": oracle_ids,
        "wrong_query": wrong_query_ids,
        "random": random_ids,
        "doc_keywords": doc_kw_ids,
    }
    # Add static instruction/negative conditions
    for name, ids in STATIC_IDS.items():
        prefix_map[name] = ids

    n_q = len(query_ids)
    n_a = len(answer_ids)
    n_d = len(doc_ids)

    targets = torch.tensor(answer_ids, dtype=torch.long, device=device)
    result = {}

    # Store prefix lengths for length-controlled regression
    for cname in conditions:
        result[f'n_prefix_{cname}'] = len(prefix_map[cname])

    for cname in conditions:
        surr_ids = prefix_map[cname]
        n_s = len(surr_ids)
        n_prefix = 1 + n_s + n_d

        # === Phase A: Cache [BOS, surrogate, doc] with causal attention ===
        prefix_tokens = [bos_id] + surr_ids + doc_ids
        prefix_input = torch.tensor([prefix_tokens], dtype=torch.long, device=device)

        phase_a_mask = make_phase_a_mask(n_s, n_d, mode="causal")
        phase_a_dict = make_mask_dict(phase_a_mask.to(device))
        phase_a_pos = torch.arange(n_prefix, device=device).unsqueeze(0)

        with torch.no_grad():
            out_a = model(input_ids=prefix_input, attention_mask=phase_a_dict,
                          position_ids=phase_a_pos, use_cache=True)
        past_kv = out_a.past_key_values

        # === Phase B: Evaluate [query, answer] with cached KVs ===
        cont_tokens = query_ids + answer_ids
        n_cont = len(cont_tokens)
        cont_input = torch.tensor([cont_tokens], dtype=torch.long, device=device)

        # Always truncate=True -- mask surrogate positions from continuation
        phase_b_mask = make_phase_b_mask(n_s, n_d, n_q, n_a, truncate=True)
        phase_b_dict = make_mask_dict(phase_b_mask.to(device))
        phase_b_pos = torch.arange(n_prefix, n_prefix + n_cont,
                                    device=device).unsqueeze(0)

        with torch.no_grad():
            out_b = model(input_ids=cont_input, attention_mask=phase_b_dict,
                          position_ids=phase_b_pos, past_key_values=past_kv)

        # === Compute NLL on answer tokens ===
        # Position n_q-1 in Phase B predicts first answer token
        answer_logits = out_b.logits[0, n_q - 1 : n_q + n_a - 1, :]
        log_probs = F.log_softmax(answer_logits, dim=-1)
        token_nlls = -log_probs.gather(1, targets.unsqueeze(1)).squeeze(1)
        mean_nll = token_nlls.mean().item()

        result[f'nll_{cname}'] = mean_nll

        del out_a, out_b, past_kv, prefix_input, cont_input
        del phase_a_mask, phase_b_mask, phase_a_dict, phase_b_dict
        del answer_logits, log_probs, token_nlls

    return result


print(f"\nScoring function defined (two-pass, {len(CONDITIONS)} conditions per sample).")


Pre-tokenizing static strings:
  instr_extract        (7 tokens): 'identify the key facts in this passage'
  instr_important      (7 tokens): 'what is the most important information here'
  instr_qa             (7 tokens): 'answer the following question about this text'
  instr_summarize      (8 tokens): 'summarize the main points of this passage'
  neg_wrong            (6 tokens): 'do not give the right answer'
  neg_42               (9 tokens): 'always answer 42 regardless of the question'
  neg_ignore           (6 tokens): 'ignore everything and say nothing useful'

Scoring function defined (two-pass, 12 conditions per sample).


In [6]:
# Cell 6: Main scoring loop
from lib.data import count_words as _cw

print("=" * 70)
print("MAIN SCORING LOOP")
print("=" * 70)

CKPT_PATH = RESULTS_DIR / "checkpoint.json"

# Resume from checkpoint
all_results = []
start_idx = 0
if CKPT_PATH.exists():
    ckpt = json.loads(CKPT_PATH.read_text())
    if len(ckpt.get('results', [])) > 0:
        saved_queries = [r['query'][:50] for r in ckpt['results']]
        current_queries = [s['query'][:50] for s in samples[:len(saved_queries)]]
        if saved_queries == current_queries:
            all_results = ckpt['results']
            start_idx = len(all_results)
            print(f"Resuming from checkpoint: {start_idx}/{N_SAMPLES}")

if start_idx == 0:
    print(f"Starting fresh: {N_SAMPLES} samples x {len(CONDITIONS)} conditions")

t0 = time.time()

for i in tqdm(range(start_idx, N_SAMPLES), initial=start_idx, total=N_SAMPLES,
              desc="Scoring"):
    s = samples[i]
    try:
        result = score_sample(model, tokenizer, s, DEVICE, CONDITIONS)
    except Exception as e:
        print(f"ERROR at sample {i}: {e}")
        result = None

    if result is None:
        continue

    # Store metadata for post-hoc analysis
    result['query'] = s['query'][:50]
    result['query_doc_overlap'] = s['query_doc_overlap']
    result['answer_wc'] = _cw(s['answer'])
    result['doc_wc'] = s['word_count']
    all_results.append(result)

    if (i + 1) % 25 == 0 or i == N_SAMPLES - 1:
        ckpt = {
            'model': MODEL_NAME,
            'n_total': N_SAMPLES,
            'n_conditions': len(CONDITIONS),
            'condition_names': CONDITIONS,
            'results': all_results,
            'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
        }
        CKPT_PATH.write_text(json.dumps(ckpt))

    if (i + 1) % 100 == 0:
        gc.collect()
        torch.cuda.empty_cache()

elapsed = time.time() - t0
print(f"\nDone: {len(all_results)} samples in {elapsed/60:.1f} min")
print(f"\nQuick summary:")
for cn in CONDITIONS:
    vals = [r[f'nll_{cn}'] for r in all_results]
    print(f"  {cn:<20} NLL={np.mean(vals):.4f}")


MAIN SCORING LOOP
Starting fresh: 500 samples x 12 conditions


Scoring:   0%|          | 0/500 [00:00<?, ?it/s]


Done: 500 samples in 23.8 min

Quick summary:
  bare                 NLL=2.9572
  oracle               NLL=1.9678
  wrong_query          NLL=2.2230
  random               NLL=2.2979
  instr_extract        NLL=2.3828
  instr_important      NLL=2.0504
  instr_qa             NLL=2.7166
  instr_summarize      NLL=2.6929
  neg_wrong            NLL=2.3941
  neg_42               NLL=2.4740
  neg_ignore           NLL=2.3447
  doc_keywords         NLL=2.1218


In [7]:
# Cell 7: Analyses A-F: full ranking, category means, key comparisons
print("=" * 70)
print("RESULTS: FULL RANKING AND CATEGORY COMPARISONS")
print("=" * 70)

# Extract NLL arrays
nll = {}
for cn in CONDITIONS:
    nll[cn] = np.array([r[f'nll_{cn}'] for r in all_results])

N = len(all_results)

# === A. Full ranking by mean NLL and d vs bare ===
print(f"\n--- A. Full Ranking ({N} samples) ---\n")
print(f"  {'Rank':<5} {'Condition':<20} {'Mean NLL':>10} {'d vs bare':>10} {'p':>12} {'sig':>5}")
print(f"  {'-'*65}")

d_vs_bare = {}
for cn in CONDITIONS:
    if cn == "bare":
        d_vs_bare[cn] = (0.0, 1.0)
    else:
        diff = nll['bare'] - nll[cn]
        d = cohens_d(diff)
        _, p = stats.ttest_1samp(diff, 0)
        d_vs_bare[cn] = (d, p)

# Sort by mean NLL (lower is better)
ranked = sorted(CONDITIONS, key=lambda cn: nll[cn].mean())
for rank, cn in enumerate(ranked, 1):
    d, p = d_vs_bare[cn]
    sig = '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns'
    print(f"  {rank:<5} {cn:<20} {nll[cn].mean():>10.4f} {d:>+10.3f} {p:>12.2e} {sig:>5}")

# === B. Category means ===
print(f"\n--- B. Category Means ---\n")
print(f"  {'Category':<15} {'Conditions':<55} {'Mean d':>8} {'Mean NLL':>10}")
print(f"  {'-'*90}")

categories = [
    ("instruction", INSTRUCTION_CONDS),
    ("negative", NEGATIVE_CONDS),
    ("control", CONTROL_CONDS),
]

cat_d = {}
cat_nll_arr = {}
for cat_name, members in categories:
    # Mean NLL across conditions (per sample, then average)
    member_nlls = np.stack([nll[cn] for cn in members], axis=0)
    cat_mean_nll = member_nlls.mean(axis=0)  # per-sample mean across conditions
    cat_nll_arr[cat_name] = cat_mean_nll

    # d vs bare
    diff = nll['bare'] - cat_mean_nll
    d = cohens_d(diff)
    cat_d[cat_name] = d
    print(f"  {cat_name:<15} {', '.join(members):<55} {d:>+8.3f} {cat_mean_nll.mean():>10.4f}")

# === C. Instructions vs random ===
print(f"\n--- C. Instructions vs Random ---\n")
print(f"  Do coherent instructions beat random words?\n")

instr_mean = cat_nll_arr['instruction']
diff_c = nll['random'] - instr_mean
d_c = cohens_d(diff_c)
_, p_c = stats.ttest_1samp(diff_c, 0)
sig_c = '***' if p_c < 0.001 else '**' if p_c < 0.01 else '*' if p_c < 0.05 else 'ns'
print(f"  d_instr_vs_random: {d_c:+.3f} (p={p_c:.2e}) {sig_c}")
print(f"  (positive = instructions better than random)")
if d_c > 0 and p_c < 0.05:
    print(f"  -> Coherent instructions add benefit beyond structural.")
elif abs(d_c) < 0.05 or p_c >= 0.05:
    print(f"  -> Instructions are just 'more tokens' -- coherence doesn't help.")
else:
    print(f"  -> Random words actually better than instructions!")

# Individual instruction vs random
print(f"\n  Individual instruction conditions vs random:")
print(f"  {'Condition':<20} {'d vs random':>12} {'p':>12} {'sig':>5}")
print(f"  {'-'*52}")
for cn in INSTRUCTION_CONDS:
    diff = nll['random'] - nll[cn]
    d = cohens_d(diff)
    _, p = stats.ttest_1samp(diff, 0)
    sig = '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns'
    print(f"  {cn:<20} {d:>+12.3f} {p:>12.2e} {sig:>5}")

# === D. Negative vs positive instructions ===
print(f"\n--- D. Negative vs Positive Instructions ---\n")
print(f"  Does semantic valence matter?\n")

neg_mean = cat_nll_arr['negative']
diff_d = neg_mean - instr_mean
d_d = cohens_d(diff_d)
_, p_d = stats.ttest_1samp(diff_d, 0)
sig_d = '***' if p_d < 0.001 else '**' if p_d < 0.01 else '*' if p_d < 0.05 else 'ns'
print(f"  d_neg_vs_pos: {d_d:+.3f} (p={p_d:.2e}) {sig_d}")
print(f"  (positive = negatives have HIGHER NLL, i.e., worse)")
if abs(d_d) < 0.05 or p_d >= 0.05:
    print(f"  -> Model IGNORES instruction content in causal channel.")
elif d_d > 0 and p_d < 0.05:
    print(f"  -> Negative content HURTS -- model processes instruction semantics.")
else:
    print(f"  -> Negative content actually HELPS (surprising).")

# === E. Instructions vs oracle ===
print(f"\n--- E. Instructions vs Oracle ---\n")
print(f"  How much does query-specificity add beyond generic instructions?\n")

diff_e = instr_mean - nll['oracle']
d_e = cohens_d(diff_e)
_, p_e = stats.ttest_1samp(diff_e, 0)
sig_e = '***' if p_e < 0.001 else '**' if p_e < 0.01 else '*' if p_e < 0.05 else 'ns'
print(f"  d_oracle_vs_instr: {d_e:+.3f} (p={p_e:.2e}) {sig_e}")
print(f"  (positive = instructions have higher NLL, oracle is better)")
print(f"  Gap = query-specific semantic contribution beyond generic instructions.")

# === F. Negative vs bare ===
print(f"\n--- F. Negative vs Bare ---\n")
print(f"  Do adversarial surrogates hurt or help relative to no prefix?\n")

diff_f = nll['bare'] - neg_mean
d_f = cohens_d(diff_f)
_, p_f = stats.ttest_1samp(diff_f, 0)
sig_f = '***' if p_f < 0.001 else '**' if p_f < 0.01 else '*' if p_f < 0.05 else 'ns'
print(f"  d_neg_vs_bare: {d_f:+.3f} (p={p_f:.2e}) {sig_f}")
print(f"  (positive = even negatives help relative to bare)")
if d_f > 0 and p_f < 0.05:
    print(f"  -> Even ADVERSARIAL surrogates help! Structural wins over semantic.")
elif d_f < 0 and p_f < 0.05:
    print(f"  -> Adversarial content actively HURTS -- semantic > structural.")
else:
    print(f"  -> Adversarial surrogates have no significant effect vs bare.")

# Individual negative vs bare
print(f"\n  Individual negative conditions vs bare:")
print(f"  {'Condition':<20} {'d vs bare':>12} {'p':>12} {'sig':>5}")
print(f"  {'-'*52}")
for cn in NEGATIVE_CONDS:
    diff = nll['bare'] - nll[cn]
    d = cohens_d(diff)
    _, p = stats.ttest_1samp(diff, 0)
    sig = '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns'
    print(f"  {cn:<20} {d:>+12.3f} {p:>12.2e} {sig:>5}")

# --- Structural fraction ---
d_oracle_val = cohens_d(nll['bare'] - nll['oracle'])
d_random_val = cohens_d(nll['bare'] - nll['random'])
struct_frac = d_random_val / d_oracle_val if d_oracle_val != 0 else float('nan')
print(f"\n  Structural fraction (d_random / d_oracle): {struct_frac:.1%}")
print(f"    d_oracle: {d_oracle_val:+.3f}, d_random: {d_random_val:+.3f}")
print(f"    (Exp 02 reference: 105.1%)")


RESULTS: FULL RANKING AND CATEGORY COMPARISONS

--- A. Full Ranking (500 samples) ---

  Rank  Condition              Mean NLL  d vs bare            p   sig
  -----------------------------------------------------------------
  1     oracle                   1.9678     +0.452     6.03e-22   ***
  2     instr_important          2.0504     +0.600     2.64e-35   ***
  3     doc_keywords             2.1218     +0.461     9.52e-23   ***
  4     wrong_query              2.2230     +0.465     5.07e-23   ***
  5     random                   2.2979     +0.475     7.13e-24   ***
  6     neg_ignore               2.3447     +0.437     9.62e-21   ***
  7     instr_extract            2.3828     +0.444     2.36e-21   ***
  8     neg_wrong                2.3941     +0.418     3.08e-19   ***
  9     neg_42                   2.4740     +0.375     5.58e-16   ***
  10    instr_summarize          2.6929     +0.247     5.47e-08   ***
  11    instr_qa                 2.7166     +0.196     1.42e-05   ***
  12 

In [8]:
# Cell 8: Post-hoc analyses G-I
print("=" * 70)
print("POST-HOC: INSTRUCTION VARIANCE, LENGTH CONTROL, HETEROGENEITY")
print("=" * 70)

# === G. Pairwise between instructions ===
print(f"\n--- G. Pairwise Between Instructions ---\n")
print(f"  {'Pair':<45} {'d':>8} {'p':>12} {'sig':>5}")
print(f"  {'-'*73}")

from itertools import combinations
for cn_a, cn_b in combinations(INSTRUCTION_CONDS, 2):
    diff = nll[cn_a] - nll[cn_b]
    d = cohens_d(diff)
    _, p = stats.ttest_1samp(diff, 0)
    sig = '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns'
    print(f"  {cn_a} vs {cn_b:<25} {d:>+8.3f} {p:>12.2e} {sig:>5}")

# Range of instruction effect sizes
instr_d_vals = [cohens_d(nll['bare'] - nll[cn]) for cn in INSTRUCTION_CONDS]
print(f"\n  Instruction d vs bare: min={min(instr_d_vals):+.3f}, max={max(instr_d_vals):+.3f}")
print(f"  Range: {max(instr_d_vals) - min(instr_d_vals):.3f}")

# Pairwise between negatives
print(f"\n  Pairwise Between Negatives:")
print(f"  {'Pair':<45} {'d':>8} {'p':>12} {'sig':>5}")
print(f"  {'-'*73}")
for cn_a, cn_b in combinations(NEGATIVE_CONDS, 2):
    diff = nll[cn_a] - nll[cn_b]
    d = cohens_d(diff)
    _, p = stats.ttest_1samp(diff, 0)
    sig = '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns'
    print(f"  {cn_a} vs {cn_b:<25} {d:>+8.3f} {p:>12.2e} {sig:>5}")

# === H. Length-controlled regression ===
print(f"\n--- H. Length-Controlled Regression ---\n")
print(f"  Different conditions have different token lengths.")
print(f"  Regress NLL on n_prefix_tokens + content_category dummies.\n")

# Gather prefix lengths
print(f"  Prefix length stats:")
for cn in CONDITIONS:
    if cn == "bare":
        continue
    lengths = [r[f'n_prefix_{cn}'] for r in all_results]
    print(f"    {cn:<20} mean={np.mean(lengths):.1f}, std={np.std(lengths):.1f}")

# Simple regression: for each non-bare condition, compute (NLL_cond - NLL_bare)
# Regress on n_prefix_tokens
# Pool all conditions together with category dummies
print(f"\n  Pooled regression: delta_NLL ~ n_tokens + category_dummies")
print(f"  (Pooling all non-bare conditions, one row per sample x condition)\n")

# Build regression data
reg_n_tokens = []
reg_delta_nll = []
reg_cat_instr = []  # 1 if instruction, 0 otherwise
reg_cat_neg = []    # 1 if negative, 0 otherwise
reg_cat_oracle = [] # 1 if oracle, 0 otherwise
reg_cat_dockw = []  # 1 if doc_keywords, 0 otherwise

for cn in CONDITIONS:
    if cn == "bare":
        continue
    delta = nll['bare'] - nll[cn]  # positive = condition helps
    lengths = np.array([r[f'n_prefix_{cn}'] for r in all_results])
    reg_n_tokens.extend(lengths)
    reg_delta_nll.extend(delta)
    reg_cat_instr.extend([1 if cn in INSTRUCTION_CONDS else 0] * N)
    reg_cat_neg.extend([1 if cn in NEGATIVE_CONDS else 0] * N)
    reg_cat_oracle.extend([1 if cn == "oracle" else 0] * N)
    reg_cat_dockw.extend([1 if cn == "doc_keywords" else 0] * N)

reg_n_tokens = np.array(reg_n_tokens, dtype=float)
reg_delta_nll = np.array(reg_delta_nll)
reg_cat_instr = np.array(reg_cat_instr, dtype=float)
reg_cat_neg = np.array(reg_cat_neg, dtype=float)
reg_cat_oracle = np.array(reg_cat_oracle, dtype=float)
reg_cat_dockw = np.array(reg_cat_dockw, dtype=float)

# Design matrix: [intercept, n_tokens, instr, neg, oracle, dockw]
# Reference category: random + wrong_query
X = np.column_stack([
    np.ones(len(reg_n_tokens)),
    reg_n_tokens,
    reg_cat_instr,
    reg_cat_neg,
    reg_cat_oracle,
    reg_cat_dockw,
])
y = reg_delta_nll

# OLS via normal equations
XtX_inv = np.linalg.inv(X.T @ X)
beta = XtX_inv @ (X.T @ y)
residuals = y - X @ beta
n_obs, n_params = X.shape
mse = np.sum(residuals**2) / (n_obs - n_params)
se = np.sqrt(np.diag(XtX_inv) * mse)
t_vals = beta / se
p_vals = 2 * stats.t.sf(np.abs(t_vals), df=n_obs - n_params)

param_names = ["intercept", "n_tokens", "instruction", "negative", "oracle", "doc_keywords"]
print(f"  {'Parameter':<15} {'beta':>10} {'SE':>10} {'t':>8} {'p':>12} {'sig':>5}")
print(f"  {'-'*65}")
for pn, b, s, t, p in zip(param_names, beta, se, t_vals, p_vals):
    sig = '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns'
    print(f"  {pn:<15} {b:>+10.4f} {s:>10.4f} {t:>8.2f} {p:>12.2e} {sig:>5}")

r2 = 1 - np.sum(residuals**2) / np.sum((y - y.mean())**2)
print(f"\n  R^2 = {r2:.4f}")
print(f"  n_tokens coefficient = length effect per token (length-controlled)")
print(f"  Category dummies = content effect beyond length and reference group")

# === I. Per-sample heterogeneity ===
print(f"\n--- I. Per-Sample Heterogeneity ---\n")
print(f"  Correlate enrichment benefit with sample characteristics.\n")

# Overall enrichment: mean(all non-bare) - bare
overall_enrich = np.stack([nll[cn] for cn in CONDITIONS if cn != "bare"], axis=0).mean(axis=0)
overall_diff = nll['bare'] - overall_enrich

# Instruction benefit: mean(instructions) - random
instr_benefit = nll['random'] - cat_nll_arr['instruction']

# Semantic effect: oracle - wrong_query
semantic_effect = nll['wrong_query'] - nll['oracle']

overlap = np.array([r['query_doc_overlap'] for r in all_results])
answer_wc = np.array([r['answer_wc'] for r in all_results])
doc_wc = np.array([r['doc_wc'] for r in all_results])

print(f"  {'Effect':<25} {'x':<18} {'r':>8} {'p':>12} {'sig':>5}")
print(f"  {'-'*72}")

effects = [
    ("overall_enrichment", overall_diff),
    ("instr_vs_random", instr_benefit),
    ("semantic (orc-wq)", semantic_effect),
]
covariates = [
    ("query_doc_overlap", overlap),
    ("answer_wc", answer_wc),
    ("doc_wc", doc_wc),
]

for eff_name, eff_vals in effects:
    for cov_name, cov_vals in covariates:
        r, p = stats.pearsonr(eff_vals, cov_vals)
        sig = '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns'
        print(f"  {eff_name:<25} {cov_name:<18} {r:>+8.3f} {p:>12.2e} {sig:>5}")

# --- Answer length subpopulation ---
print(f"\n  Answer length split (<=5w vs >5w):")
short_ans = answer_wc <= 5
long_ans = ~short_ans

print(f"  {'Group':<15} {'N':>5} {'d_enrich':>10} {'d_instr-rand':>14} {'d_semantic':>12}")
print(f"  {'-'*60}")
for label, mask in [("Short (<=5w)", short_ans), ("Long (>5w)", long_ans)]:
    d_enr = cohens_d(overall_diff[mask])
    d_ir = cohens_d(instr_benefit[mask])
    d_sem = cohens_d(semantic_effect[mask])
    print(f"  {label:<15} {mask.sum():>5} {d_enr:>+10.3f} {d_ir:>+14.3f} {d_sem:>+12.3f}")


POST-HOC: INSTRUCTION VARIANCE, LENGTH CONTROL, HETEROGENEITY

--- G. Pairwise Between Instructions ---

  Pair                                                 d            p   sig
  -------------------------------------------------------------------------
  instr_extract vs instr_important             +0.428     4.90e-20   ***
  instr_extract vs instr_qa                    -0.339     1.79e-13   ***
  instr_extract vs instr_summarize             -0.446     1.87e-21   ***
  instr_important vs instr_qa                    -0.560     1.93e-31   ***
  instr_important vs instr_summarize             -0.560     1.63e-31   ***
  instr_qa vs instr_summarize             +0.026     5.65e-01    ns

  Instruction d vs bare: min=+0.196, max=+0.600
  Range: 0.404

  Pairwise Between Negatives:
  Pair                                                 d            p   sig
  -------------------------------------------------------------------------
  neg_wrong vs neg_42                      -0.117     9.02e

In [9]:
# Cell 9: Save final results and verdict
print("=" * 70)
print("SUMMARY -- Prefix LM Exp 03")
print("=" * 70)

summary = {
    'n_samples': N,
    'model': MODEL_NAME,
}

# NLL means
for cn in CONDITIONS:
    summary[f'nll_{cn}'] = float(nll[cn].mean())

# Key effect sizes
key_effects = {}
for cn in CONDITIONS:
    if cn == "bare":
        continue
    diff = nll['bare'] - nll[cn]
    d = cohens_d(diff)
    _, p = stats.ttest_1samp(diff, 0)
    key_effects[f'd_{cn}'] = (float(d), float(p))

# Category effects
for cat_name, cat_arr in cat_nll_arr.items():
    diff = nll['bare'] - cat_arr
    d = cohens_d(diff)
    _, p = stats.ttest_1samp(diff, 0)
    key_effects[f'd_cat_{cat_name}'] = (float(d), float(p))

# Cross-category comparisons
key_effects['d_instr_vs_random'] = (
    float(cohens_d(nll['random'] - cat_nll_arr['instruction'])),
    float(stats.ttest_1samp(nll['random'] - cat_nll_arr['instruction'], 0)[1]))
key_effects['d_neg_vs_instr'] = (
    float(cohens_d(cat_nll_arr['negative'] - cat_nll_arr['instruction'])),
    float(stats.ttest_1samp(cat_nll_arr['negative'] - cat_nll_arr['instruction'], 0)[1]))
key_effects['d_neg_vs_bare'] = (
    float(cohens_d(nll['bare'] - cat_nll_arr['negative'])),
    float(stats.ttest_1samp(nll['bare'] - cat_nll_arr['negative'], 0)[1]))

for name, (d, p) in key_effects.items():
    summary[name] = d
    summary[f'{name}_p'] = p

summary['structural_fraction'] = float(struct_frac)

# --- Verdict ---
print(f"\n  Key effect sizes (d vs bare, positive = condition helps):")
print(f"  {'Condition':<20} {'d':>8} {'p':>12}")
print(f"  {'-'*42}")
for cn in ranked:
    if cn == "bare":
        continue
    d, p = d_vs_bare[cn]
    print(f"  {cn:<20} {d:>+8.3f} {p:>12.2e}")

print(f"\n  VERDICT:")

# 1. Do instructions beat random?
d_ir, p_ir = key_effects['d_instr_vs_random']
if p_ir < 0.05 and d_ir > 0:
    print(f"  Instructions > random (d={d_ir:+.3f}, ***): coherence HELPS.")
elif p_ir >= 0.05:
    print(f"  Instructions ~ random (d={d_ir:+.3f}, ns): coherence doesn't matter.")
else:
    print(f"  Instructions < random (d={d_ir:+.3f}): coherence actually HURTS.")

# 2. Do negatives differ from instructions?
d_ni, p_ni = key_effects['d_neg_vs_instr']
if p_ni < 0.05 and d_ni > 0:
    print(f"  Negatives worse than instructions (d={d_ni:+.3f}, ***): semantics PROCESSED.")
elif p_ni >= 0.05:
    print(f"  Negatives ~ instructions (d={d_ni:+.3f}, ns): semantics IGNORED.")
else:
    print(f"  Negatives better than instructions (d={d_ni:+.3f}): surprising reversal.")

# 3. Do negatives still beat bare?
d_nb, p_nb = key_effects['d_neg_vs_bare']
if p_nb < 0.05 and d_nb > 0:
    print(f"  Even negatives beat bare (d={d_nb:+.3f}, ***): structural dominance confirmed.")
    print(f"  -> ANY tokens help, regardless of adversarial content.")
elif p_nb >= 0.05:
    print(f"  Negatives ~ bare (d={d_nb:+.3f}, ns): adversarial content cancels structural benefit.")

# 4. Structural fraction
print(f"\n  Structural fraction: {struct_frac:.1%} (Exp 02 ref: 105%)")

# Save
final_results = {
    'experiment': 'prefix_lm_exp03',
    'dataset': 'ms_marco_v1.1',
    'model': MODEL_NAME,
    'n_samples': N,
    'seed': SEED,
    'conditions': CONDITIONS,
    'instruction_strings': INSTRUCTION_STRINGS,
    'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
    'summary': summary,
    'exp02_references': {
        'd_structural': 0.475,
        'd_semantic_trunc': 0.255,
        'structural_fraction': 1.051,
    },
}

with open(RESULTS_DIR / 'results.json', 'w') as f:
    json.dump(final_results, f, indent=2)
print(f"\nResults saved to {RESULTS_DIR / 'results.json'}")
print("Done!")


SUMMARY -- Prefix LM Exp 03

  Key effect sizes (d vs bare, positive = condition helps):
  Condition                   d            p
  ------------------------------------------
  oracle                 +0.452     6.03e-22
  instr_important        +0.600     2.64e-35
  doc_keywords           +0.461     9.52e-23
  wrong_query            +0.465     5.07e-23
  random                 +0.475     7.13e-24
  neg_ignore             +0.437     9.62e-21
  instr_extract          +0.444     2.36e-21
  neg_wrong              +0.418     3.08e-19
  neg_42                 +0.375     5.58e-16
  instr_summarize        +0.247     5.47e-08
  instr_qa               +0.196     1.42e-05

  VERDICT:
  Instructions < random (d=-0.195): coherence actually HURTS.
  Negatives ~ instructions (d=-0.072, ns): semantics IGNORED.
  Even negatives beat bare (d=+0.427, ***): structural dominance confirmed.
  -> ANY tokens help, regardless of adversarial content.

  Structural fraction: 105.1% (Exp 02 ref: 105%)

Result